# Heisig Character Decomposition & Mnemonic Generator

Type any Chinese/Japanese character below to see its component breakdown and generate a mnemonic story.

**To use:** Run the cell below (click `▶` or press Shift+Enter), then use the interactive form that appears.

In [1]:
import json, os, sys
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets

sys.path.insert(0, os.path.join(os.getcwd(), 'heisig_addon'))
from llm import generate_story

with open('heisig_addon/data/heisig_data.json', encoding='utf-8') as f:
    data = json.load(f)

CSS = """
<style>
.hc { font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
  background: #fafafa; border-radius: 12px; padding: 24px;
  max-width: 480px; margin: 12px 0; border: 1px solid #e0e0e0; text-align: center; }
.hc .ch { font-size: 96px; line-height: 1.2; }
.hc .kw { font-size: 28px; font-weight: 600; color: #1a1a2e; margin: 8px 0; }
.hc .nm { font-size: 13px; color: #888; }
.hc .dt { font-size: 15px; color: #555; margin: 4px 0; text-align: left; }
.hc .lb { font-weight: 600; color: #333; }
.hc .st { font-style: italic; color: #333; margin-top: 8px; text-align: left; line-height: 1.5; }
.hc hr { border: none; border-top: 1px solid #e0e0e0; margin: 12px 0; }
</style>
"""

Q = '"'

def card_html(char, story=None):
    info = data.get(char)
    if not info:
        return f'<div class={Q}hc{Q}><p style={Q}color:#999;padding:20px{Q}>Character <b>{char}</b> not found ({len(data):,} characters available).</p></div>'
    nums = [f'{l} #{info[k]}' for k, l in [('RSH_number','RSH'),('RTH_number','RTH'),('RTK_number','RTK')] if info.get(k)]
    kw = info.get('keyword', '')
    h = [f'<div class={Q}hc{Q}><div class={Q}ch{Q}>{char}</div><div class={Q}kw{Q}>{kw}</div>']
    if nums:
        joined = ', '.join(nums)
        h.append(f'<div class={Q}nm{Q}>{joined}</div>')
    h.append('<hr>')
    for key, label in [('reading','Reading'),('components_detail','Components'),('ids','IDS'),('spatial','Layout'),('decomposition','Decomposition')]:
        val = info.get(key, '')
        if val:
            h.append(f'<div class={Q}dt{Q}><span class={Q}lb{Q}>{label}:</span> {val}</div>')
    if story:
        h.append(f'<hr><div class={Q}st{Q}>{story}</div>')
    h.append('</div>')
    return ''.join(h)

char_in = widgets.Text(placeholder='e.g. 学, 森, 愛, 休', description='Character:', style={'description_width': '80px'})
api_in = widgets.Password(placeholder='Free key from aistudio.google.com/apikey', description='Gemini key:', style={'description_width': '80px'}, layout=widgets.Layout(width='450px'))
dec_btn = widgets.Button(description='Decompose', button_style='primary', icon='search')
story_btn = widgets.Button(description='Generate Story', button_style='success', icon='magic-wand')
out = widgets.Output()

def do_decompose(_=None):
    c = list(char_in.value.strip() or ' ')[0]
    if c == ' ':
        return
    with out:
        clear_output(wait=True)
        display(HTML(CSS + card_html(c)))

def do_story(_=None):
    c = list(char_in.value.strip() or ' ')[0]
    if c == ' ':
        return
    info = data.get(c)
    if not info:
        do_decompose()
        return
    key = api_in.value.strip() or os.environ.get('GEMINI_API_KEY', '')
    if not key:
        msg = 'Enter a Gemini API key above. Get a free one at <a href="https://aistudio.google.com/apikey" target="_blank">aistudio.google.com/apikey</a>'
        with out:
            clear_output(wait=True)
            display(HTML(CSS + card_html(c, story=msg)))
        return
    with out:
        clear_output(wait=True)
        display(HTML(CSS + card_html(c, story='Generating story...')))
    s = generate_story(c, info, 'gemini', key, 'gemini-2.0-flash')
    with out:
        clear_output(wait=True)
        display(HTML(CSS + card_html(c, story=s)))

dec_btn.on_click(do_decompose)
story_btn.on_click(do_story)
char_in.on_submit(do_decompose)

display(widgets.VBox([
    widgets.HBox([char_in, dec_btn, story_btn]),
    api_in,
    out
]))
print(f'Ready - {len(data):,} characters loaded')

/var/folders/5k/d6h2188d25lcr62fshmd956r0000gn/T/ipykernel_39411/4026953989.py:87: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  char_in.on_submit(do_decompose)


Ready - 5,397 characters loaded
